In [1]:
from models.neural_nets import FeedForwardNN, PolynomialNN, PolynomialNN_relu
from models.pi_nets import CCP, CCP_relu, CCP_relu_poly
from data.polynomials import PolynomialGenerator
import pytorch_lightning as pl
import torch
%reload_ext autoreload
%autoreload 2

c:\Users\Bruger\miniconda3\envs\thesis\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
polynomial = lambda x, y: 5*x**2 + 6*y**2
polyname = '5x2+6y2_noise'
in_dim = polynomial.__code__.co_argcount
from torch.distributions.normal import Normal
dataloader = PolynomialGenerator(polynomial, 100000, in_dim, Normal(0,5), Normal(0,1))
experiment_name = "two_variable"

#model = CCP(in_dim, 64, 5, 1)
#model = CCP_relu(2, 64, 5, 1)
#model = CCP_relu_poly(in_dim, 64, 2, 1)
#model = FeedForwardNN(in_dim, [64, 32, 16], 1)
model = PolynomialNN(in_dim, [64], 1, 2)
#model = PolynomialNN_relu(in_dim, [64,64,64], 1, 2)

In [8]:
lambda_t = 'lambda x, y: x**2+3*y'
l = eval(lambda_t, globals())

In [9]:
l(5,3)

34

: 

In [6]:
logger = pl.loggers.TensorBoardLogger("tb_logs", name=f"{experiment_name + '/' + type(model).__name__}/{polyname}")
logger.log_hyperparams({"data_dist": str(dataloader.data_dist), "noise": str(dataloader.noise)})
trainer = pl.Trainer(limit_train_batches=64,max_epochs=30, log_every_n_steps=25, logger=logger)
trainer.fit(model=model, datamodule=dataloader)
trainer.logger.finalize("success")
trainer.logger.save()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 257   
1 | loss   | MSELoss    | 0     
--------------------------------------
257       Trainable params
0         Non-trainable params
257       Total params
0.001     Total estimated model params size (MB)


c:\Users\Bruger\miniconda3\envs\thesis\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0:  63%|██████▎   | 431/689 [00:16<00:09, 26.82it/s, loss=1.32e+05, v_num=0]

c:\Users\Bruger\miniconda3\envs\thesis\lib\site-packages\pytorch_lightning\trainer\call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
